$\color{brown}{\text{HCIN 620 Lab 5 Unsupervised Clustering Algorithms}}$

The following resources have been used to generate this notebook

https://realpython.com/k-means-clustering-python/

Data is located at UCI repository. Check the link for further information. 

https://archive.ics.uci.edu/ml/datasets/HCV+data

The k-means clustering method is an unsupervised machine learning technique used to identify clusters of data objects in a dataset. There are many different types of clustering methods, but k-means is one of the oldest and most approachable. These traits make implementing k-means clustering in Python reasonably straightforward, even for novice programmers and data scientists.

If you’re interested in learning how and when to implement k-means clustering in Python, then this is the right place. You’ll walk through an end-to-end example of k-means clustering using Python, from preprocessing the data to evaluating results.

In this tutorial, you’ll learn:
 
* What k-means clustering is

* When to use k-means clustering to analyze your data

* How to implement k-means clustering in Python with scikit-learn

* How to select a meaningful number of clusters </div>

Notebook by Reza Afra, Ph.D. and Barbara Berkovich, Ph.D., M.A.

Last update December 29, 2020


# **Step 1 Environment Setup**

##Import Code Libraries.

The first set of libraries is the same as Modules 2-4 with the addition of [plotly.express ](https://plotly.com/python/plotly-express/)

**Run Import 1.**


In [ ]:
##  Import 1. Data manipulation and visualization libraries

import pandas as pd             # data manipulation library
import io                       # Helps with Input/output of csv files
import numpy as np              # Fast array operations    
import matplotlib.pyplot as plt # a library to plot nice graphs
plt.style.use('ggplot') 
import seaborn as sns           # plot style and color on next line
sns.set(style="white", palette="muted", color_codes=True) 

import plotly.express as px     # for EDA data scattermatrix visualization

print('Import 1 complete')

Scikit Learn (Rhymes with psychic) is the main machine learning library in Python. The [scikit-learn.org](https://scikit-learn.org/stable/) maintains the open source code. </br>

**Question 1**  Give a very brief description of the following sklearn classes*:

1. KMeans   [Type answer here] 
2. PCA      [Type answer here]
3. Pipeline [Type answer here] 

'* *Note that classes in this context refers to subprograms within the code libaries. Do not confuse this with the term classes as it is used in machine learning classification models.*


**Run the code to complete the import 2.**

In [ ]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score


print('Import 2 complete')

# **Step 2 Data Cleaning**

**Use the Folder and upload icons (left) to upload data-lab-5-hcv.csv**

**Run the code** to read the hepatitis C virus (HCV) data in the Colab Runtime Environment.


In [ ]:
hcv = pd.read_csv("data-lab-5-hcv.csv")
print('Data connection complete')

**Question 2** Another way to view the data in Python is simply to use the name of the data frame.  In this case we've read the input file into a data frame call **hcv**. Enter that name on the next command line. </br>
**Run it.**

In [ ]:
# Answer 2: Add the name of the data frame below



##Handle missing values



Run the code below to check for null values. 

In [ ]:
hcv.isnull().sum()

**Question 3** </br>

Edit this text cell to answer the questions below: 
1. List the variables with NaN (not a number) values. [Type Answer Here]
2. List the ALP row numbers that contain NaN          [Type Answer Here]



**Question 4**
Use the fillna command using the 'ffill' method to replace NaN with the value of the previous row. </br>Print the data frame as  you did in Question 2 to confirm that NaN has been replaced.

**Run the fillna step**

In [ ]:
# Answer 4: Add code below to fill the missing values.


**Question 5** In the original dataset (Question 2), lines 613 and 614 had missing values (NaN) for the variable ALP. Answer the following questions about the results of the 'ffill' method.


1. What is the new values of ALP on rows 613 and 614? 
   
   [Type answer here]
2. Where did that number come from? 
   
   [Type answer here]
3. How does this differ from the backfill method used in Lab 2 (Question 11)?
 
   [type answer here]

4.  What is the term we use when we fill the missing values with "guesses"?
   [type answer here]

5.  What is another way we could handle the missing data?
    [type answer here]





##Drop Unnecessary Variables

**Question 6** The first column,'Unnamed:  0' is a dummy variable in place of the patient id (removed for privacy).</br>Enter and run the code to drop the column 'Unnamed: 0'</br>Print the dataframe as you have in previous steps to verify that the NaN values have been replaced.</br>

NOTE: Python has context sensitive help. Type in **hcv.dr** and pause to see the help in a pop-up window.
</br>




In [ ]:
# Answer 6:  Drop the columns 'Unnamed: 0' and 'Category' 





# **Step 3 Exploratory Data Analysis (EDA)**


##Scatter Matrix

**Category** contains the code indicating the diagnosis.
0=Blood donor (undiagnosed)
0s=suspect blood donor (suspected of liver disease, unconfirmed)
1=Hepatitis
2=Fibrosis
3=Cirrhosis
If we were using a supervised machine learning technique, this column might be the target and the values would be training labels. </br>
Since we are using and unsupervised technique, we will just use Category in our visual data exploration, and not in the actual clustering algorithm.

Read about the plotly.express [scatter_matrix](https://plotly.com/python/splom/)

**Run the plotly command below which plots the correlations between the labs for each value of Category.**


In [ ]:
# Let's explore the relationship between features using the plotly.express (px) library

fig = px.scatter_matrix(hcv,
    dimensions=list(hcv.drop(columns="Category").columns),
    color="Category")
fig.show()

**Question 7**  </br>
Study the scatter matrix. Can you see cells where the data is well separated, and would be a good candidate for clustering? Edit this text box to record your answers below.

1. List the  columns where the category colors *are* well separated across multiple rows.

[Type answer here]

2. List the columns where the category colors *are not* well separated across multiple rows. 

[Type answer here]



Since we are working with a multidimensional array, we can only visualize the location of dots on a plane. We can zoom in on a single cell of the scatter matrix to see more detail.

**Run the next code cell** to zoom in on the plot of CHE vs CHOL.

In [ ]:
##Scatter matrix on CHE, CHOL only 
fig = px.scatter_matrix(hcv,      
    dimensions=['CHE', 'CHOL'],
    color='Category')
fig.show()

**Question 8**


With the increased resolution in these plots, how might you interpret the relationship between CHE, CHOL and the category information?

[Type answer here]

There are no fixed rules for visualizing the relationships within this data. One can iterate two by two matricies for other columns as we have just done.
</br></br>
*At the  beginnning of any analysis do a little homework to increase your clinical understanding of the data elements. Whenever possible seek collaboration with a trained clinician to be aware of known correlations.*
</br></br>

##Heatmap

**Run the code below** to generate a heatmap



In [ ]:
# Look at correlations more closely
corr = hcv.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

##Correlation Matrix

**Question 9**

Enter the command to generate a correlation matrix.

**Run the corr command**

In [ ]:
# Answer 9: Type the command to print correlation matrix in this cell, and run.


##Preprocessing



##Encode the data


Since there is no ordinal relationship among the category labels, the following code encodes these as separate columns of ones and zeros.

**Run the encoding.**


In [ ]:
#Encoding 
true_label_names = hcv['Category']

label_encoder = LabelEncoder()
true_labels = label_encoder.fit_transform(true_label_names)
hcv = pd.get_dummies(hcv, dtype=float)
hcv

**Question 10** The original Category column has been transformed.

How many category columns are there now?

**Answer 10**

[Type answer here]

##Pipeline



In practical machine learning pipelines, it’s common for the data to undergo multiple sequences of transformations before it feeds into a clustering algorithm. You learned about the importance of one of these transformation steps, feature scaling in an earlier lab. An equally important data transformation technique is dimensionality reduction, which reduces the number of features in the dataset by either removing or combining them.
Dimensionality reduction techniques help to address a problem with machine learning algorithms known as the curse of dimensionality. In short, as the number of features increases, the feature space becomes sparse. This sparsity makes it difficult for algorithms to find data objects near one another in higher-dimensional space.

###Dimensionality Reduction using Principal Component Analysis (PCA)
PCA is one of many dimensionality reduction techniques. It transforms the input data by projecting it into a lower number of dimensions called components. The components capture the variability of the input data through a linear combination of the input data’s features.

The next code block is another application of scikit-learn pipelines. In Lab 4, we used a pipeline to scale the variables, and assign to the x and y values for regression. In this lab, we will use the pipeline to scale the numeric data and perform PCA dimensionality reduction in preparation for the clustering algorithm.




###MinMaxScaler

The pipeline will implement an alternative to the StandardScaler class called MinMaxScaler for feature scaling. You use MinMaxScaler when you do not assume that the shape of all your features follows a normal distribution.

**Run the pipeline.**

In [ ]:
label_encoder.classes_
n_clusters = len(label_encoder.classes_)

# Dimensionality reduction starts here
preprocessor = Pipeline(
     [
         ("scaler", MinMaxScaler()),
         ("pca", PCA(n_components=2, random_state=42)),
     ]
)

print('MinMaxScaler and PCA pipline complete')

# **Step 4 Build the Model**

##KMeans Clustering Algorithm

Now that you’ve built a pipeline to process the data, you’ll build a separate pipeline to perform k-means clustering. You’ll override the following default arguments of the KMeans class:</br>
**init=k-means++**" instead of "random" to ensure centroids are initialized with some distance between them. In most cases, this will be an improvement over "random".

**n_init=50**: You’ll increase the number of initializations to ensure you find a stable solution.

**max_iter=500**: You’ll increase the number of iterations per initialization to ensure that k-means will converge.


**Run the initialization pipeline.**

In [ ]:
clusterer = Pipeline(
    [
       (
            "kmeans",
            KMeans(
                n_clusters=n_clusters,
                init="k-means++",
                n_init=50,
                max_iter=500,
                random_state=42,
            ),
        ),
    ]
 )

print('initialization pipline complete')

**Run the preprocesser and clustering pipeline.**

In [ ]:
pipe = Pipeline(
  [
       ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
   )
print('preprocessing and clustering complete')

**Run the code to fit the model to the data.**

In [ ]:
pipe.fit(hcv)

  

##Evaluate Results


As mentioned earlier, the scale for each of these clustering performance metrics ranges from -1 to 1. A silhouette coefficient of 0 indicates that clusters are significantly overlapping one another, and a silhouette coefficient of 1 indicates clusters are well-separated. An ARI score of 0 indicates that cluster labels are randomly assigned, and an ARI score of 1 means that the true labels and predicted labels form identical clusters.

**Run the code to generate a silhouette coefficient.**

In [ ]:
preprocessed_data = pipe["preprocessor"].transform(hcv)
predicted_labels = pipe["clusterer"]["kmeans"].labels_
silhouette_score(preprocessed_data, predicted_labels)

The silhouette value indicates that the model predicted the correct cluster 94.5% of the time. 

We can add the predicted_labels column to the hcv dataframe to visually confirm how many predicted_labels match with the categories.

**Run the code to print the hcv data frame with the predicted labels.**

*Note: Scroll data table to the right to see the predicted_labels column.*

In [ ]:
hcv['predicted_labels'] = predicted_labels
hcv

Since you specified n_components=2 in the PCA step of the k-means clustering pipeline, you can also visualize the data in the context of the true labels and predicted labels. 

**Run the code to plot the results using a pandas DataFrame and the seaborn plotting library.**

In [ ]:
hcv.drop(columns='predicted_labels', inplace=True)  #drop the predicted labels before plotting

pcadf = pd.DataFrame(
     pipe["preprocessor"].transform(hcv),
    columns=["component_1", "component_2"],
 )

pcadf["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_
pcadf["true_label"] = label_encoder.inverse_transform(true_labels)

plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
     x="component_1",
     y="component_2",
     s=50,
     data=pcadf,
     hue="predicted_cluster",
     style="true_label",
     palette="Set2",
 )

scat.set_title(
    "Clustering results from HCV Data"
 )
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
plt.show()

The visual representation of the clusters confirms the results of the two clustering evaluation metrics. The performance of your pipeline was pretty good. The clusters only slightly overlapped, and cluster assignments were much better than random.